# Phase 1: データ品質確認と基礎統計

## 分析目的
ANALYSIS_PLAN.md の Phase 1 に従い、以下を実施：
1. データ品質チェック（欠損値、回答一貫性、外れ値）
2. 記述統計量の算出
3. Page_IDマッチング確認

## 判断基準
- 欠損率>20%の項目は解析から除外を検討
- 極端な偏りがある項目は変数変換を検討

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. データ読み込み

In [ ]:
# データファイルパス
data_dir = '../data/analysis/'

# データ読み込み
before_df = pd.read_csv(data_dir + 'before_excel_compliant.csv')
after_df = pd.read_csv(data_dir + 'after_excel_compliant.csv')
comment_df = pd.read_csv(data_dir + 'comment.csv')

print("=== データ読み込み完了 ===")
print(f"授業前データ: {before_df.shape}")
print(f"授業後データ: {after_df.shape}")
print(f"感想データ: {comment_df.shape}")

## 2. データ構造確認

In [ ]:
print("=== 授業前データ構造 ===")
print(before_df.info())
print("\n=== 授業前データ サンプル ===")
display(before_df.head())

print("\n=== 授業後データ構造 ===")
print(after_df.info())
print("\n=== 授業後データ サンプル ===")
display(after_df.head())

print("\n=== 感想データ構造 ===")
print(comment_df.info())
print("\n=== 感想データ サンプル ===")
display(comment_df.head())

## 3. Page_IDマッチング確認

In [ ]:
# Page_IDの一致確認
before_ids = set(before_df['Page_ID'])
after_ids = set(after_df['Page_ID'])
comment_ids = set(comment_df['page-ID']) if 'page-ID' in comment_df.columns else set(comment_df['Page_ID'])

print("=== Page_ID マッチング分析 ===")
print(f"授業前データ Page_ID数: {len(before_ids)}")
print(f"授業後データ Page_ID数: {len(after_ids)}")
print(f"感想データ Page_ID数: {len(comment_ids)}")

# 共通のPage_ID
common_ids = before_ids & after_ids
print(f"\n前後共通 Page_ID数: {len(common_ids)}")
print(f"マッチング率: {len(common_ids)/max(len(before_ids), len(after_ids))*100:.1f}%")

# マッチングしないPage_ID
before_only = before_ids - after_ids
after_only = after_ids - before_ids

if before_only:
    print(f"\n授業前のみ: {sorted(before_only)}")
if after_only:
    print(f"授業後のみ: {sorted(after_only)}")

# クラス別分布
print("\n=== クラス別分布 ===")
print("授業前:")
print(before_df['class'].value_counts().sort_index())
print("\n授業後:")
print(after_df['class'].value_counts().sort_index())

## 4. 欠損値分析

In [ ]:
def analyze_missing_values(df, title):
    """
    欠損値分析関数
    """
    print(f"\n=== {title} 欠損値分析 ===")
    
    # 欠損値の割合
    missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
    missing_counts = df.isnull().sum()
    
    missing_info = pd.DataFrame({
        '欠損数': missing_counts,
        '欠損率(%)': missing_pct
    })
    
    # 欠損値がある項目のみ表示
    missing_info = missing_info[missing_info['欠損数'] > 0]
    
    if len(missing_info) > 0:
        print(missing_info.sort_values('欠損率(%)', ascending=False))
        
        # 20%以上の欠損がある項目
        high_missing = missing_info[missing_info['欠損率(%)'] > 20]
        if len(high_missing) > 0:
            print(f"\n⚠️ 20%以上欠損の項目: {list(high_missing.index)}")
    else:
        print("欠損値なし")
    
    return missing_info

# 各データセットの欠損値分析
before_missing = analyze_missing_values(before_df, "授業前データ")
after_missing = analyze_missing_values(after_df, "授業後データ")
comment_missing = analyze_missing_values(comment_df, "感想データ")

In [ ]:
# 欠損値パターンの可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 授業前データの欠損値ヒートマップ
sns.heatmap(before_df.isnull(), ax=axes[0], cbar=True, yticklabels=False, cmap='viridis')
axes[0].set_title('授業前データ 欠損値パターン')
axes[0].set_xlabel('カラム')

# 授業後データの欠損値ヒートマップ
sns.heatmap(after_df.isnull(), ax=axes[1], cbar=True, yticklabels=False, cmap='viridis')
axes[1].set_title('授業後データ 欠損値パターン')
axes[1].set_xlabel('カラム')

plt.tight_layout()
plt.show()

## 5. Q1項目（水溶液認識）の分析

In [ ]:
# Q1項目の分析（授業前後共通）
q1_cols_before = [col for col in before_df.columns if col.startswith('Q1_')]
q1_cols_after = [col for col in after_df.columns if col.startswith('Q1_')]

print("=== Q1項目（水溶液認識）分析 ===")
print(f"授業前 Q1項目: {q1_cols_before}")
print(f"授業後 Q1項目: {q1_cols_after}")

# 前後で対応する項目の確認
def standardize_q1_columns(before_cols, after_cols):
    """
    Q1項目の対応関係を確認・標準化
    """
    print("\n=== Q1項目対応関係 ===")
    
    # 物質名を抽出して対応付け
    substances = ['Saltwater', 'Sugarwater', 'Muddywater', 'Ink', 'MisoSoup', 'SoySauce']
    
    correspondence = {}
    for substance in substances:
        before_col = None
        after_col = None
        
        for col in before_cols:
            if substance in col:
                before_col = col
                break
        
        for col in after_cols:
            if substance in col:
                after_col = col
                break
        
        if before_col and after_col:
            correspondence[substance] = {'before': before_col, 'after': after_col}
            print(f"{substance}: {before_col} ↔ {after_col}")
    
    return correspondence

q1_correspondence = standardize_q1_columns(q1_cols_before, q1_cols_after)

In [ ]:
# Q1項目の記述統計
def analyze_q1_responses(correspondence, before_df, after_df):
    """
    Q1項目の回答分析
    """
    print("\n=== Q1項目 正答率分析 ===")
    
    # 正答基準（仮定: 食塩水・砂糖水・味噌汁・醤油=True、泥水・墨汁=False）
    correct_answers = {
        'Saltwater': True,
        'Sugarwater': True, 
        'Muddywater': False,
        'Ink': False,
        'MisoSoup': True,
        'SoySauce': True
    }
    
    results = []
    
    for substance, cols in correspondence.items():
        before_col = cols['before']
        after_col = cols['after']
        correct = correct_answers.get(substance)
        
        # 授業前正答率
        before_correct = (before_df[before_col] == correct).sum()
        before_total = before_df[before_col].notna().sum()
        before_rate = before_correct / before_total * 100 if before_total > 0 else 0
        
        # 授業後正答率
        after_correct = (after_df[after_col] == correct).sum()
        after_total = after_df[after_col].notna().sum()
        after_rate = after_correct / after_total * 100 if after_total > 0 else 0
        
        results.append({
            '物質': substance,
            '正答': correct,
            '授業前_正答数': before_correct,
            '授業前_総数': before_total,
            '授業前_正答率': round(before_rate, 1),
            '授業後_正答数': after_correct,
            '授業後_総数': after_total,
            '授業後_正答率': round(after_rate, 1),
            '変化': round(after_rate - before_rate, 1)
        })
    
    results_df = pd.DataFrame(results)
    display(results_df)
    
    return results_df

q1_results = analyze_q1_responses(q1_correspondence, before_df, after_df)

## 6. 記述統計量の算出

In [ ]:
def calculate_descriptive_stats(df, title):
    """
    記述統計量の算出
    """
    print(f"\n=== {title} 記述統計量 ===")
    
    # 数値変数の記述統計
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("\n--- 数値変数 ---")
        stats_df = df[numeric_cols].describe()
        display(stats_df)
        
        # 歪度・尖度の計算
        skew_kurt = pd.DataFrame({
            '歪度': df[numeric_cols].skew(),
            '尖度': df[numeric_cols].kurtosis()
        })
        print("\n--- 歪度・尖度 ---")
        display(skew_kurt)
    
    # カテゴリ変数の度数分布
    categorical_cols = df.select_dtypes(include=['object', 'bool']).columns
    if len(categorical_cols) > 0:
        print("\n--- カテゴリ変数度数分布 ---")
        for col in categorical_cols:
            if col != 'Page_ID':  # Page_IDは除外
                print(f"\n{col}:")
                print(df[col].value_counts(dropna=False))

# 各データセットの記述統計
calculate_descriptive_stats(before_df, "授業前データ")
calculate_descriptive_stats(after_df, "授業後データ")

## 7. データ分布の可視化

In [ ]:
# Q1正答率の可視化
fig = go.Figure()

# 授業前後の正答率比較
substances = q1_results['物質'].tolist()
before_rates = q1_results['授業前_正答率'].tolist()
after_rates = q1_results['授業後_正答率'].tolist()

fig.add_trace(go.Bar(
    name='授業前',
    x=substances,
    y=before_rates,
    text=[f'{rate}%' for rate in before_rates],
    textposition='auto'
))

fig.add_trace(go.Bar(
    name='授業後',
    x=substances,
    y=after_rates,
    text=[f'{rate}%' for rate in after_rates],
    textposition='auto'
))

fig.update_layout(
    title='Q1項目 授業前後正答率比較',
    xaxis_title='物質',
    yaxis_title='正答率 (%)',
    barmode='group',
    height=600
)

fig.show()

In [ ]:
# 授業後データの評価項目分布
if 'Q4_ExperimentInterestRating' in after_df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Q4: 実験興味度
    q4_counts = after_df['Q4_ExperimentInterestRating'].value_counts().sort_index()
    axes[0,0].bar(q4_counts.index, q4_counts.values)
    axes[0,0].set_title('Q4: 実験への興味度')
    axes[0,0].set_xlabel('評価 (1:面白くない - 4:とても面白い)')
    axes[0,0].set_ylabel('人数')
    
    # Q5: 新しい学び
    if 'Q5_NewLearningsRating' in after_df.columns:
        q5_counts = after_df['Q5_NewLearningsRating'].value_counts().sort_index()
        axes[0,1].bar(q5_counts.index, q5_counts.values)
        axes[0,1].set_title('Q5: 新しい学び')
        axes[0,1].set_xlabel('評価')
        axes[0,1].set_ylabel('人数')
    
    # Q6: 理解度
    if 'Q6_DissolvingUnderstandingRating' in after_df.columns:
        q6_counts = after_df['Q6_DissolvingUnderstandingRating'].value_counts().sort_index()
        axes[1,0].bar(q6_counts.index, q6_counts.values)
        axes[1,0].set_title('Q6: 理解度')
        axes[1,0].set_xlabel('評価 (1:わからない - 4:よくわかった)')
        axes[1,0].set_ylabel('人数')
    
    # クラス分布
    class_counts = after_df['class'].value_counts().sort_index()
    axes[1,1].bar(class_counts.index, class_counts.values)
    axes[1,1].set_title('クラス別回答者数')
    axes[1,1].set_xlabel('クラス')
    axes[1,1].set_ylabel('人数')
    
    plt.tight_layout()
    plt.show()

## 8. データ品質サマリー

In [ ]:
print("\n" + "="*60)
print("Phase 1 データ品質確認 結果サマリー")
print("="*60)

print(f"\n📊 データ規模:")
print(f"・授業前回答者: {len(before_df)}名")
print(f"・授業後回答者: {len(after_df)}名")
print(f"・前後マッチング: {len(common_ids)}名 ({len(common_ids)/max(len(before_ids), len(after_ids))*100:.1f}%)")

print(f"\n🔍 データ品質:")
# 欠損値が20%以上の項目があるかチェック
high_missing_before = before_missing[before_missing['欠損率(%)'] > 20] if len(before_missing) > 0 else pd.DataFrame()
high_missing_after = after_missing[after_missing['欠損率(%)'] > 20] if len(after_missing) > 0 else pd.DataFrame()

if len(high_missing_before) == 0 and len(high_missing_after) == 0:
    print("・20%以上の欠損項目: なし ✅")
else:
    print("・20%以上の欠損項目: あり ⚠️")
    if len(high_missing_before) > 0:
        print(f"  授業前: {list(high_missing_before.index)}")
    if len(high_missing_after) > 0:
        print(f"  授業後: {list(high_missing_after.index)}")

print(f"\n📈 主要結果:")
print(f"・Q1水溶液認識項目: {len(q1_correspondence)}項目で前後比較可能")
if len(q1_results) > 0:
    avg_change = q1_results['変化'].mean()
    print(f"・平均正答率変化: {avg_change:.1f}ポイント")
    positive_changes = (q1_results['変化'] > 0).sum()
    print(f"・改善項目数: {positive_changes}/{len(q1_results)}項目")

print(f"\n✅ Phase 1 完了 - Phase 2 (統計的検証) に進行可能")
print("="*60)